# 1. 环境配置

## 1.1 python 环境准备

In [1]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3 beautifulsoup4==4.14.3 langchain_chroma==1.1.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [2]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. RAG Agent 应用开发

## 2.1 简介

在第四章里我们已经学习过如何创建并使用向量数据库（RAG），那假如我们希望从向量数据库中获取信息并应用到智能体中，我们就需要先完成第一步制作数据库后才能实现。

在完成向量数据库的创建后，我们就可以将检索的过程进行封装，然后输入就是用户提出或智能体润色后的问题，输出就是搜索到的片段信息。

当我们基于前面的代码创建出了一个向量数据库后：

In [3]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://zh.d2l.ai/chapter_introduction/index.html")
docs = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 1500,
  chunk_overlap = 150)
splits = text_splitter.split_documents(docs)

from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os
embeddings = DashScopeEmbeddings(
  dashscope_api_key=os.getenv('DASHSCOPE_API_KEY'), 
  model="text-embedding-v1")
vectordb = Chroma.from_documents(documents=splits,
  embedding=embeddings,
  persist_directory='./chroma')

USER_AGENT environment variable not set, consider setting it to identify your requests.


然后我们就可以通过 Chroma 加载向量数据库：

In [4]:
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_chroma import Chroma
import os

embeddings = DashScopeEmbeddings(
    dashscope_api_key=os.getenv("DASHSCOPE_API_KEY"), model="text-embedding-v1"
)
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma"  # 必须与创建数据库的路径一致
)

然后将提问→检索向量数据库→回复相关片段这一流程改造成工具来进行使用：

In [5]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vectordb.similarity_search(query, k=1)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

这里设置的 response_format 表示不仅把拼接后的 serialized 注入到 ToolMessage 的 content 里。同时也会把 retrieved_docs 单独保存到 ToolMessage 的 artifact 字段中。

然后我们需要将大模型、工具、提示词统一配置到 create_agent 中：

In [6]:
from langchain_community.chat_models import ChatTongyi
from langchain.agents import create_agent

llm = ChatTongyi(model="qwen-max") 

tools = [retrieve_context]

prompt = (
    "You have access to a tool that retrieves context from a deep learning book. "
    "Use the tool to help answer user queries."
)

agent = create_agent(llm, tools, system_prompt=prompt)

创建完成后我们就可以根据书本里的问题对其进行调用了：

In [7]:
query = (
    "What is the deep learning model used in computer vision tasks? "
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the deep learning model used in computer vision tasks? 
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_7279a56d83cb42d89c5e17)
 Call ID: call_7279a56d83cb42d89c5e17
  Args:
    query: deep learning model used in computer vision tasks
================================= Tool Message =================================
Name: retrieve_context

Source: {'title': '1. 引言 — 动手学深度学习 2.0.0 documentation', 'source': 'https://zh.d2l.ai/chapter_introduction/index.html', 'language': 'en'}
Content: 1.7. 特点¶
到目前为止，本节已经广泛地讨论了机器学习，它既是人工智能的一个分支，也是人工智能的一种方法。
虽然深度学习是机器学习的一个子集，但令人眼花缭乱的算法和应用程序集让人很难评估深度学习的具体成分是什么。
这就像试图确定披萨所需的配料一样困难，因为几乎每种成分都是可以替代的。
如前所述，机器学习可以使用数据来学习输入和输出之间的转换，例如在语音识别中将音频转换为文本。
在这样做时，通常需要以适合算法的方式表示数据，以便将这种表示转换为输出。
深度学习是“深度”的，模型学习了许多“层”的转换，每一层提供一个层次的表示。
例如，靠近输入的层可以表示数据的低级细节，而接近分类输出的层可以表示用于区分的更抽象的概念。
由于表示学习（representation
learning）目的是寻找表示

我们可以通过以下方式来进行创建（前提是需要创建并加载数据库）：

In [8]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """自动根据用户问题检索知识并注入到系统提示"""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vectordb.similarity_search(last_query, k=3)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    system_message = (
        "You are a helpful assistant. "
        "Use the following retrieved context to answer the user:\n\n"
        f"{docs_content}"
    )
    print(system_message)
    return system_message

创建完后，可以以中间键的形式添加到 Agent 中：

In [9]:
from langchain.agents import create_agent 

llm = ChatTongyi(model="qwen-max") 
agent = create_agent(llm, tools=[], middleware=[prompt_with_context])

然后可以正常对其进行调用，这个时候就会把检索到的信息添加到系统提示词后进行回复：

In [10]:
query = "What is computer vision?"
for step in agent.stream(
  {"messages": [{"role": "user", "content": query}]},
  stream_mode="values",
):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

What is computer vision?
You are a helpful assistant. Use the following retrieved context to answer the user:

同样，上面的列表仅仅触及了机器学习对实际应用的影响之处的皮毛。
例如，机器人学、物流、计算生物学、粒子物理学和天文学最近取得的一些突破性进展至少部分归功于机器学习。
因此，机器学习正在成为工程师和科学家必备的工具。
关于人工智能的非技术性文章中，经常提到人工智能奇点的问题：机器学习系统会变得有知觉，并独立于主人来决定那些直接影响人类生计的事情。
在某种程度上，人工智能已经直接影响到人类的生计：信誉度的自动评估，车辆的自动驾驶，保释决定的自动准予等等。
甚至，我们可以让Alexa打开咖啡机。
幸运的是，我们离一个能够控制人类创造者的有知觉的人工智能系统还很远。
首先，人工智能系统是以一种特定的、面向目标的方式设计、训练和部署的。
虽然他们的行为可能会给人一种通用智能的错觉，但设计的基础是规则、启发式和统计模型的结合。
其次，目前还不存在能够自我改进、自我推理、能够在试图解决一般任务的同时，修改、扩展和改进自己的架构的“人工通用智能”工具。
一个更紧迫的问题是人工智能在日常生活中的应用。
卡车司机和店员完成的许多琐碎的工作很可能也将是自动化的。
农业机器人可能会降低有机农业的成本，它们也将使收割作业自动化。
工业革命的这一阶段可能对社会的大部分地区产生深远的影响，因为卡车司机和店员是许多国家最常见的工作之一。
此外，如果不加注意地应用统计模型，可能会导致种族、性别或年龄偏见，如果自动驱动相应的决策，则会引起对程序公平性的合理关注。
重要的是要确保小心使用这些算法。
就我们今天所知，这比恶意超级智能毁灭人类的风险更令人担忧。

12. 计算性能
12.1. 编译器和解释器
12.2. 异步计算
12.3. 自动并行
12.4. 硬件
12.5. 多GPU训练
12.6. 多GPU的简洁实现
12.7. 参数服务器


13. 计算机视觉
13.1. 图像增广
13.2. 微调
13.3. 目标